# Projeto 8 - Detecção de Pragas com Imagens Agrícolas usando Inteligência Artificial

A  ferrugem  da  folha  do  trigo  é  a  doença  mais  comum  desta cultura.  As  perdas  em rendimento de grãos podem chegar a 50%.Essa doença  manifesta-se desde o surgimento das  primeiras folhas até a maturação da planta.  

Inicialmente,  surgem  pequenos  nódulos  arredondados,  amarelo-alaranjados,  dispostos sem  ordenação  (parecendo  ferrugem  mesmo),  normalmente  localizados  na  face  superior  das folhas, estendendo-se ao caule. Estas frutificações ficam sempre recobertas pela epiderme até o final do ciclo da planta.

O patógeno sobrevive no verão-outono parasitando plantas de trigo que se constituem na principal fonte de problemas em plantações de trigo em países mais quentes, como o Brasil. As condições ambientais para o desenvolvimento da doença são temperatura média de 20ºC e mais de 6 horas de molhamento foliar contínuo.

O  objetivo  deste  Projeto  é  construir  um  modelo  de  Deep  Learning  para  classificar corretamente se uma planta (trigo) é saudável, possui ferrugem no caule ou ferrugem na folha.

Fonte dos dados: https://zindi.africa/competitions/iclr-workshop-challenge-1-cgiar-computer-vision-for-crop-disease/data

## 1. Instalando e carregando os pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Pacote de utilitários
!pip install -q imutils

In [3]:
# Comando para silenciar o Keras
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [4]:
# Imports

# Pacotes para manipulação e visualização de dados
import os
import cv2
import pickle
import imutils
import random
import sklearn
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from imutils import paths
from random import randint
from collections import defaultdict

# Pacotes para Deep Learning
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model, save_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.optimizers import Adam 

# Pacotes para processamento de dados e avaliação do modelo
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Define a área de plotagem das imagens
from IPython.display import clear_output
plt.rcParams['figure.figsize'] = (15, 9)
%matplotlib inline

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 08 - Detecção de Pragas com Imagens Agrícolas" --iversions

Author: Projeto 08 - Detecção de Pragas com Imagens Agrícolas

cv2       : 4.7.0
sklearn   : 1.0.2
keras     : 2.11.0
matplotlib: 3.5.2
numpy     : 1.21.5
tensorflow: 2.11.0
imutils   : 0.5.4



## 2. Carga das imagens de treino

In [9]:
# Imagens de treino
imagens_treino = "dados/train"

# Imagens de teste
imagens_teste = "dados/test"

In [10]:
# Caminho das imagens
caminho_imagens_treino = sorted(list(paths.list_images(imagens_treino)))

In [11]:
# Suffle das imagens
random.shuffle(caminho_imagens_treino)

In [14]:
# Lista para armazenar as imagens
dados = []

# Lista para armazenar os labels
labels = []

# Dimensões das imagens
image_dims = (224, 224, 3)

In [15]:
# Loop pelos caminhos das imagens de treino

print("\nIniciando o processamento das imagens de treino. Aguarde.")

count = 0

for caminho in caminho_imagens_treino:
    
    # Leitura da imagem
    image = cv2.imread(caminho)
    
    # Redimensionamento
    # Argumento cv2.INTER_AREA é para realizar a interpolação das imagens
    # Isso é necessário porque nem todas as imagens podem ficar com 224 x 224
    image = cv2.resize(image, (image_dims[1], image_dims[0]), cv2.INTER_AREA)
    
    # Converte a imagem para array numpy
    image = img_to_array(image)
    
    # Adiciona o array da imagem à lista de arrays
    dados.append(image)

    # Extrai o label das classes
    label = caminho.split(os.path.sep)[-2]

    # Adiciona o label à lista de labels
    labels.append(label)
    
    # Atualiza o contador
    count += 1

print("\nProcessamento Concluído. Total de imagens processadas:", count)


Iniciando o processamento das imagens de treino. Aguarde.

Processamento Concluído. Total de imagens processadas: 562


In [16]:
# Padroniza a intensidade dos pixels para o range [0,1]
dados = np.array(dados, dtype = 'float') / 255.0

In [17]:
# Convertemos os labels para o formato de array numpy
labels = np.array(labels)

In [18]:
# Conversão dos labels em binário
# Amostra dos labels
labels[1:10]

array(['stem_rust', 'stem_rust', 'stem_rust', 'stem_rust',
       'healthy_wheat', 'stem_rust', 'stem_rust', 'leaf_rust',
       'stem_rust'], dtype='<U13')

In [19]:
# Cria o binarizador
binarizador = LabelBinarizer()

In [20]:
# Aplica o binarizador aos labels
labels = binarizador.fit_transform(labels)

In [21]:
# Visualiza os labels
labels[1:10]

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [22]:
# Salva o binarizador, para usar nos dados de teste
obj_binarizador = open("modelos/binarizador.pickle", "wb")
obj_binarizador.write(pickle.dumps(binarizador))
obj_binarizador.close()

In [23]:
# Divisão dos dados em treino e teste
(X_treino, X_teste, y_treino, y_teste) = train_test_split(dados, labels, test_size = 0.2)

In [24]:
# Shape dos dados de treino e teste
print("Shape de X_treino:", X_treino.shape)
print("Shape de y_treino:", y_treino.shape)
print("Shape de X_teste:", X_teste.shape)
print("Shape de y_teste:", y_teste.shape)

Shape de X_treino: (449, 224, 224, 3)
Shape de y_treino: (449, 3)
Shape de X_teste: (113, 224, 224, 3)
Shape de y_teste: (113, 3)


## 3. Construção do modelo